In [ ]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import re
from splinter.exceptions import ElementDoesNotExist

from config import password
from sqlalchemy import create_engine

csv_path = "Resources/Amzon_All_Books.csv"

# Mac Users

In [ ]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

In [ ]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
url = 'https://www.amazon.com/s?i=stripbooks&s=relevanceexprank&Adv-Srch-Books-Submit.x=35&Adv-Srch-Books-Submit.y=8&unfiltered=1&ref=sr_adv_b'
browser.visit(url)


In [ ]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
results = []

In [ ]:
def scrapeWebPage():
    try:
       
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        parents = soup.select("div[data-asin]")   
        for parent in parents:
            my_dict = {}
            my_dict["title"] = parent.select(".a-color-base.a-text-normal")[0].text
            rating = parent.select(".a-icon-alt")
            
            if len(rating) == 0:
                 my_dict["rating"] = 0
            else:
                my_dict["rating"] =  rating[0].text
            author = parent.select(".a-color-secondary .a-size-base+ .a-size-base")
            #.a-color-secondary .a-size-base.a-link-normal
            #.a-color-secondary .a-size-base+ .a-size-base
            if(len(author)== 0):
                my_dict["author"] = 0
            else:
                my_dict["author"] =  author[0].text.strip()

            review_count = parent.select(".a-size-small .a-link-normal .a-size-base")
            if(len(review_count)== 0):
                my_dict["review count"] = 0
            else:
                my_dict["review count"] =  int(review_count[0].text.replace(',',''))
            results.append(my_dict)
            
            notLastPage = True
        if(browser.find_by_css(".a-last a")):
            browser.find_by_css(".a-last a").click()
        else:
            notLastPage = False
        return notLastPage
    except Exception as e:
        print(f"Exception !!!  {e}")
        
notLastPage = True
forcedExit = 0
while notLastPage:
    notLastPage = scrapeWebPage()
#     forcedExit += 1
#     if forcedExit == 2:
#         notLastPage = False

In [ ]:
print(len(results))
print(type(results))

In [ ]:
amazon_all_books_db = pd.DataFrame(results)
amazon_all_books_db.columns = ['author', 'rating', 'no_of_reviews', 'title']

# Set index to book_id
amazon_all_books_db.index.name = 'book_id'

amazon_all_books_db.head(50)

# Writing to csv

In [ ]:
amazon_all_books_db.to_csv(csv_path,index=False,header=True)

# Data Base 

In [ ]:
# Create Postgres engine
engine = create_engine('postgresql://postgres:'+ password + '@localhost:5432/Amazon_Books')



In [ ]:
# Read tables
engine.table_names()

In [ ]:
# Insert data into postgres table
amazon_all_books_db.to_sql(name='all_books', con=engine, if_exists='append', index=True)